# LSTM

Test of simple MLP models with different amount of neurons, hidden layers and size of input vector

## Load Data

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
df = pd.read_csv("data_N_6000_noise_level_0.01.csv")
df['Time'] = 1
# df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [4]:
# df_diff.head(10)

In [5]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        self.X_, self.y_ = self.__make_stack(dataframe)
        print(self.X_.shape)
        print(self.y_.shape)

    def __len__(self):
        return len(self.X_)

    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]

    def __make_stack(self, df: pd.DataFrame):
        seq_amount = df.shape[0] - 2 * self.n_dots - 1
        X = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        y = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        for i in range(seq_amount):
            X[i, :, :] = torch.reshape(torch.tensor(df.values[i:i+self.n_dots, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))
            y[i, :, :] = torch.reshape(torch.tensor(df.values[i+1:i+1+self.n_dots, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))

        # stacks = [[df.iloc[:-self.n_dots]]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return (X, y)

In [6]:
N_DOTS = 20
BATCH_SIZE = 1
N_DOT_PARAMETERS = 4
N_LSTM_LAYERS = 1

In [7]:
# dataset = SequentDataset(df_diff[0:5000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
# test = SequentDataset(df_diff[5000:6000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset = SequentDataset(df[0:5000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df[5000:6000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)

torch.Size([4499, 250, 4])
torch.Size([4499, 250, 4])
torch.Size([499, 250, 4])
torch.Size([499, 250, 4])


In [8]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=N_DOTS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=N_DOTS, shuffle=False)
test_loader = DataLoader(test, batch_size=N_DOTS, shuffle=False)

## Create Model

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=4, num_layers=10):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x = x.to(device)
        batch_size = x.size(0)
        self.h = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        self.h = self.h.to(device)
        self.c = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        self.c = self.c.to(device)
        out, _ = self.lstm(x, (self.h, self.c))

        # print(out.size())

        # Pass through fully connected layer
        out = self.linear(out)  # We want the output corresponding to the last time step
        return out


In [10]:
model = LSTMModel(input_size=N_DOT_PARAMETERS, hidden_layer_size = N_DOTS, output_size = 4, num_layers=N_LSTM_LAYERS).to(device)

# loss_model = nn.MSELoss(reduction='mean')
loss_model = nn.L1Loss(reduction='mean')

opt = Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.95)

In [11]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loss = []
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)
        loss = loss_model(y_pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)

    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/250], train_loss = 1.3847, val_loss = 1.1542
Epoch [2/250], train_loss = 0.8165, val_loss = 0.4318
Epoch [3/250], train_loss = 0.3190, val_loss = 0.2105
Epoch [4/250], train_loss = 0.1709, val_loss = 0.1264
Epoch [5/250], train_loss = 0.1062, val_loss = 0.0815
Epoch [6/250], train_loss = 0.0713, val_loss = 0.0600
Epoch [7/250], train_loss = 0.0567, val_loss = 0.0545
Epoch [8/250], train_loss = 0.0539, val_loss = 0.0525
Epoch [9/250], train_loss = 0.0454, val_loss = 0.0418
Epoch [10/250], train_loss = 0.0398, val_loss = 0.0408
Epoch [11/250], train_loss = 0.0376, val_loss = 0.0335
Epoch [12/250], train_loss = 0.0355, val_loss = 0.0343
Epoch [13/250], train_loss = 0.0348, val_loss = 0.0368
Epoch [14/250], train_loss = 0.0356, val_loss = 0.0367
Epoch [15/250], train_loss = 0.0338, val_loss = 0.0310
Epoch [16/250], train_loss = 0.0300, val_loss = 0.0286
Epoch [17/250], train_loss = 0.0310, val_loss = 0.0390
Epoch [18/250], train_loss = 0.0335, val_loss = 0.0289
Epoch [19/250], tra

In [21]:
start, _  = test[0]
# print(start.size())
# start = torch.reshape(start, (1, start.size(0)))
predictions = []
model.eval()

for X, y in test_loader:
    with torch.no_grad():
        for i in range(N_DOTS, 500):
            y_pred = model(X)
            predictions.append(y_pred[0, -1, :])
            X[0, :-1, :] = X.clone()[0, 1:, :]
            X[0, -1, :] = y_pred[0, -1, :]

    break

In [13]:
len(predictions)

250

In [22]:
p_x = pd.DataFrame(np.vstack(list(map(lambda x: x.cpu().detach().numpy(), predictions))), columns=["Time", "X", "Y", "Z"])
_, y  = test[:500]
p_y = pd.DataFrame(y.cpu().detach().numpy()[:, 0, :], columns=["Time", "X", "Y", "Z"])

# p_x = p_x.cumsum()
# p_y = p_y.cumsum()

In [23]:
p_x["C"] = "Predict"
p_y["C"] = "True"
p_z = pd.DataFrame(start, columns=["Time", "X", "Y", "Z"])
p_z["C"] = "Start"
d = pd.concat([p_x,p_y, p_z],axis=0)


In [24]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C',markers=0)
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()